In [19]:
%load_ext autoreload
%autoreload 2

# Tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd
import geopandas as gpd
import numpy as np
import pickle
pd.set_option('display.max_columns', None) # para poder visualizar todas las columnas de los dfFrames

from tqdm import tqdm

# Ignorar los warnings
# -----------------------------------------------------------------------
import warnings
warnings.filterwarnings('ignore')

# Librería para el acceso a variables y funciones
# -----------------------------------------------------------------------
import sys
sys.path.append("../")
from src import soporte_rentabilidad as sr
from src import soporte_yolo as sy
from src import soporte_scoring as ss

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
paths_transformers = ["../transformers/target_encoder.pkl", "../transformers/scaler.pkl", "../transformers/model.pkl"]
df = sr.predecir_alquiler("../data/transformed/final_sale.pkl", paths_transformers)

In [3]:
df.head(1)

,codigo,precio,precio_por_zona,tipo,exterior,planta,ascensor,tamanio,habitaciones,banios,aire_acondicionado,trastero,terraza,patio,estado,direccion,descripcion,anunciante,contacto,urls_imagenes,geometry,distrito,alquiler_predicho
0,104792745,149900.0,1180.0,piso,True,4,True,127.0,3,2,False,False,False,True,good,"carretera de Huesca, 21","Junto a la Academia General Militar, en carret...",FINCAS RUIZ,876210884,['https://img4.idealista.com/blur/WEB_LISTING-...,POINT (-0.86935 41.6973),Distrito Rural,1180.285


In [14]:
df = sy.identificar_urls_habitaciones(df, 'urls_imagenes')

100%|██████████| 365/365 [14:45<00:00,  2.43s/it]


Se han eliminado 93 filas, donde 'url_cocina' o 'url_banio' son None o NaN.


In [32]:
df[df['anunciante']=='ND']

,codigo,precio,precio_por_zona,tipo,exterior,planta,ascensor,tamanio,habitaciones,banios,aire_acondicionado,trastero,terraza,patio,estado,direccion,descripcion,anunciante,contacto,urls_imagenes,geometry,distrito,alquiler_predicho,url_cocina,url_banio
86,106386368,148500.0,1954.0,piso,True,7,True,76.0,3,1,True,False,True,False,good,"plaza de Santo Domingo, 13",ABSTENERSE DE LLAMAR INMOBILIARIAS Y SIMILARES...,ND,645084847,['https://img4.idealista.com/blur/WEB_LISTING-...,POINT (-0.88941 41.65781),Casco Histórico,904.526954,https://img4.idealista.com/blur/WEB_LISTING-M/...,https://img4.idealista.com/blur/WEB_LISTING-M/...
87,106774867,149495.0,1372.0,piso,True,en,True,109.0,2,1,False,True,False,False,good,"calle de Mosén Pedro Dosset, 10","Magnifico piso junto al mercado central, con 2...",ND,654097795,['https://img4.idealista.com/blur/WEB_LISTING-...,POINT (-0.88392 41.65667),Casco Histórico,852.214084,https://img4.idealista.com/blur/WEB_LISTING-M/...,https://img4.idealista.com/blur/WEB_LISTING-M/...
98,106918117,134000.0,2436.0,piso,True,4,False,55.0,2,1,True,False,False,False,good,"calle del Río Duero, 30","Esta encantadora y acogedora vivienda, con un ...",ND,ND,['https://img4.idealista.com/blur/WEB_LISTING-...,POINT (-0.89903 41.65904),La Almozara,685.705000,https://img4.idealista.com/blur/WEB_LISTING-M/...,https://img4.idealista.com/blur/WEB_LISTING-M/...
101,106527664,110000.0,1746.0,piso,True,1,False,63.0,1,1,False,False,True,True,good,"calle Bartolomé Llorente, 24",Se vende espectacular piso con terraza de 40 m...,ND,ND,['https://img4.idealista.com/blur/WEB_LISTING-...,POINT (-0.92565 41.65338),Oliver-Valdefierro,819.531667,https://img4.idealista.com/blur/WEB_LISTING-M/...,https://img4.idealista.com/blur/WEB_LISTING-M/...
112,106205923,99000.0,1650.0,piso,True,2,False,60.0,3,1,False,False,False,False,good,"calle de Andrés Vicente, 26",¡Oportunidad Única en Las Delicias! Descubre e...,ND,722424964,['https://img4.idealista.com/blur/WEB_LISTING-...,POINT (-0.91419 41.64914),Delicias,720.058333,https://img4.idealista.com/blur/WEB_LISTING-M/...,https://img4.idealista.com/blur/WEB_LISTING-M/...
115,106487680,145000.0,1208.0,piso,True,1,False,120.0,3,2,False,True,True,True,good,"calle Gaspar de Pex, 8",IDEAL PARA FAMILIAS. Sancho Abogados de Zarago...,ND,627135937,['https://img4.idealista.com/blur/WEB_LISTING-...,POINT (-0.9719 41.70065),Distrito Rural,1260.335000,https://img4.idealista.com/blur/WEB_LISTING-M/...,https://img4.idealista.com/blur/WEB_LISTING-M/...
121,106973452,130000.0,1857.0,piso,True,2,False,70.0,3,1,True,False,False,False,good,"avenida de Logroño, 48","Vendo piso REFORMADO en el centro de Casetas, ...",ND,625684407,['https://img4.idealista.com/blur/WEB_LISTING-...,POINT (-1.02819 41.71906),Distrito Rural,839.660504,https://img4.idealista.com/blur/WEB_LISTING-M/...,https://img4.idealista.com/blur/WEB_LISTING-M/...
127,106802882,80000.0,800.0,casa de campo,ND,ND,ND,100.0,2,1,True,False,True,False,good,"Urbanización los Guarales, 77","luz, agua, piscina, frutales, cochera, traster...",ND,651683111,['https://img4.idealista.com/blur/WEB_LISTING-...,POINT (-0.7968 41.72315),Distrito Rural,838.532500,https://img4.idealista.com/blur/WEB_LISTING-M/...,https://img4.idealista.com/blur/WEB_LISTING-M/...
128,106970728,129000.0,2224.0,piso,True,bj,False,58.0,2,1,True,False,False,False,good,Barrio Arrabal,Vivienda reformada completamente a estrenar. 2...,ND,ND,['https://img4.idealista.com/blur/WEB_LISTING-...,POINT (-0.87327 41.66645),El Rabal,734.150000,https://img4.idealista.com/blur/WEB_LISTING-M/...,https://img4.idealista.com/blur/WEB_LISTING-M/...
130,106436024,75000.0,882.0,piso,True,-1,True,85.0,1,1,False,False,True,False,good,"calle del Lucero del Alba, 19","Precioso Loft, aunque en escrituras figura com...",ND,ND,['https://img4.idealista.com/blur/WEB_LISTING-...,POINT (-0.92703 41.64183),Oliver-Valdefierro,828.558712,https://img4.idealista.com/blur/WEB_LISTING-M/...,https://img4.idealista.com/blur/WEB_LISTING-M/..

In [33]:
df_sample = df.head(1)

In [35]:
df_sample = ss.analizar_propiedades(df_sample)

Analizando propiedades:   0%|          | 0/1 [00:00<?, ?it/s]

In [36]:
df_sample

,codigo,precio,precio_por_zona,tipo,exterior,planta,ascensor,tamanio,habitaciones,banios,aire_acondicionado,trastero,terraza,patio,estado,direccion,descripcion,anunciante,contacto,urls_imagenes,geometry,distrito,alquiler_predicho,url_cocina,url_banio,puntuacion_cocina,puntuacion_banio,mts_cocina,mts_banio
0,104792745,149900.0,1180.0,piso,True,4,True,127.0,3,2,False,False,False,True,good,"carretera de Huesca, 21","Junto a la Academia General Militar, en carret...",FINCAS RUIZ,876210884,['https://img4.idealista.com/blur/WEB_LISTING-...,POINT (-0.86935 41.6973),Distrito Rural,1180.285,https://img4.idealista.com/blur/WEB_LISTING-M/...,https://img4.idealista.com/blur/WEB_LISTING-M/...,7.0,7.0,12.0,6.0


In [16]:
df.to_pickle("../data/transformed/final_tags.pkl")